In [ ]:
from __future__ import print_function
import tensorflow.keras as keras

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras import backend as K
import requests
# import pandas as pd
from numpy import genfromtxt
import zipfile
import numpy as np
import matplotlib.pyplot as plt

# input image dimensions
img_rows, img_cols = 28, 28
batch_size = 128
num_classes = 10
epochs = 12

# load mnist method 1
def download_unzip(url):
    print("[1-1] Download MNIST from scidm.nchc.org.tw")
    local_filename = url.split('/')[-1]
    r = requests.get(url)

    # NOTE the stream=True parameter below
    #with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    print("[Finish] Downloaded")
    print("[1-2] unzip file")
    with zipfile.ZipFile(local_filename, 'r') as zip_ref:
        zip_ref.extractall("./")
    print("[Finish] generating mnist_test.csv & mnist_train.csv")
    return local_filename

f_mnist_in_csv_zip = "https://scidm.nchc.org.tw/dataset/10b7845f-8f66-4055-9b6b-8f320c2ef2ae/resource/1cb5ee3b-3b1d-4cc8-bfd9-be340000dae6/nchcproxy/fashionmnist.zip"

download_unzip(f_mnist_in_csv_zip)

print("[Step 3] loading mnist file into data structure ")

mnist_test = genfromtxt("fashion-mnist_test.csv", delimiter=',')    
mnist_train = genfromtxt("fashion-mnist_train.csv", delimiter=',')    

x_train= mnist_train[1:,1:]
y_train= mnist_train[1:,0:1]
x_test = mnist_test[1:,1:]
y_test = mnist_test[1:,0:1]

x_train = x_train.reshape(len(x_train), 28, 28,1)
x_test = x_test.reshape(len(x_test), 28, 28,1)
input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print("x_train =shape=> {}".format(x_train.shape))
print("y_train =shape=> {}".format(y_train.shape))
print("x_test =shape=> {}".format(x_test.shape))
print("y_test =shape=> {}".format(y_test.shape))

print("[Finish] converted ")

print("[step 4] build model ")
# convert class vectors to binary class matrices

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
print("[Finish] keras model success")
print("[step 5] evaluate...")
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("[done] ")